In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd

from os import path
import os
import glob
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt

from IPython.display import clear_output
from IPython.display import display

from collections import defaultdict
from datetime import datetime

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 800)

import warnings
warnings.filterwarnings('ignore')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [27]:
#Source address data
data = pd.read_csv('/Users/michaelwilliams/Documents/MGH T32/Research/MHB/Analysis/MHB_Walkability/Analytic Dataset/Address_Data/Post_Geocode/MyHealthyBrain_Addresses_PostGeocode.csv')
print(data.shape)
display(data.head(2))

#List layers (feature classes) inside the geodatabase
print(gpd.list_layers('/Users/michaelwilliams/Documents/MGH T32/Research/MHB/Analysis/MHB_Walkability/Analytic Dataset/WalkabilityIndex/Natl_WI.gdb'))

#Load a Walkability Index layer
gdf = gpd.read_file('/Users/michaelwilliams/Documents/MGH T32/Research/MHB/Analysis/MHB_Walkability/Analytic Dataset/WalkabilityIndex/Natl_WI.gdb', layer="NationalWalkabilityIndex")

#2019 Census tract shapefile
tract_2019_shp = gpd.read_file('/Users/michaelwilliams/Documents/NEU PhD/Research/SDOHE/Data/Census/Geography/Shapefiles/cbg/2019/nhgis0003_shapefile_tl2019_us_blck_grp_2019.zip')
tract_2019_shp = tract_2019_shp.to_crs('EPSG:4326')
print(tract_2019_shp.crs)

(60, 86)


,ObjectID,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgComp,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,CntryName,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,MatchID,StrucType,StrucDet,IN_SingleLine,USER_id,USER_redcap_event_name,USER_caide_sex,USER_caide_age,USER_caide_education,USER_caide_hypertension,USER_caide_cholesterol,USER_caide_height,USER_caide_height_meters,USER_caide_weight,USER_caide_weight_kg,USER_caide_bmi_calculation,USER_caide_bmi,USER_caide_physical_activity,USER_caide_score,USER_tics_total,USER_faq_total,USER_address,USER_address_geocode,USER_initial_inquiry_v2,USER_waitlist_referral_source,USER_id_MHB
0,1,M,100.0,A,"5241 S Teresa Pt, Homosassa, Florida, 34446","5241 S Teresa Pt, Homosassa, FL, 34446, USA",5241 S Teresa Pt,PointAddress,NaN,NaN,"5241 S Teresa Pt, Homosassa, Florida, 34446",NaN,NaN,20.0,NaN,5241,NaN,NaN,NaN,NaN,S,NaN,Teresa,Pt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5241 S Teresa Pt,NaN,NaN,NaN,NaN,Homosassa,NaN,Citrus County,Florida,FL,NaN,NaN,34446.0,2435.0,USA,United States,ENG,0,-82.527652,28.786060,-82.527387,28.785972,-82.528387,-82.526387,28.784972,28.786972,NaN,AQHAfAEAAdABAJUBBAAAAAAMElNPVVRIIFRFUkVTQSBQT0lOVBUENTI0MWYDRU5H,Residential,Single Family,"5241 South Teresa Point Homosassa, FL 34446",16,screening_ii_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5241 South Teresa Point Homosassa, FL 34446","5241 South Teresa Point Homosassa, FL 34446",14,4.0,16_MHB
1,2,M,100.0,A,"410 Miller Ln, Rochester, New York, 14617","410 Miller Ln, Rochester, NY, 14617, USA",410 Miller Ln,PointAddress,NaN,NaN,"410 Miller Ln, Rochester, New York, 14617",NaN,NaN,20.0,NaN,410,NaN,NaN,NaN,NaN,NaN,NaN,Miller,Ln,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,410 Miller Ln,NaN,NaN,NaN,NaN,Rochester,NaN,Monroe County,New York,NY,NaN,NaN,14617.0,4311.0,USA,United States,ENG,0,-77.596270,43.205257,-77.596008,43.205264,-77.597008,-77.595008,43.204264,43.206264,NaN,AQEDkAEAVQgCAJUBBAAAAAAMC01JTExFUiBMQU5FFQM0MTBmA0VORw,NaN,NaN,410 Miller Lane Rochester New York 14617,11,screening_ii_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,410 Miller Lane Rochester New York 14617,410 Miller Lane Rochester New York 14617,14,5.0,11_MHB


                       name geometry_type
0  NationalWalkabilityIndex  MultiPolygon
EPSG:4326


In [ ]:
## Quick data checks on post-geocoded address data
# record 88 unmatched, address field was blank
# record 86 at score == 84. Usually i do som evaluation on addressess below 85, but since this is so close and its only 1, we are probably safe to leve it. 
#data[data['Score'] < 85] #

#data[~data["Match_type"].isin(["A", "T"])]

#data[data['Status'] != 'M']

,ObjectID,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgComp,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,CntryName,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,MatchID,StrucType,StrucDet,IN_SingleLine,USER_id,USER_redcap_event_name,USER_caide_sex,USER_caide_age,USER_caide_education,USER_caide_hypertension,USER_caide_cholesterol,USER_caide_height,USER_caide_height_meters,USER_caide_weight,USER_caide_weight_kg,USER_caide_bmi_calculation,USER_caide_bmi,USER_caide_physical_activity,USER_caide_score,USER_tics_total,USER_faq_total,USER_address,USER_address_geocode,USER_initial_inquiry_v2,USER_waitlist_referral_source,USER_id_MHB
16,17,U,0.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,88,screening_ii_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,88_MHB


In [28]:
#Spatial Join
#construct geoDataframe
geodata = gpd.GeoDataFrame(data,
    geometry=gpd.points_from_xy(data.X, data.Y),
    crs=4326)

data_sj = gpd.sjoin(geodata, tract_2019_shp, how='left', predicate='within')

data_sj.head()
data_sj["GEOID"].astype(str).str.len().value_counts()


GEOID
12    59
3      1
Name: count, dtype: int64

In [30]:
data_sj.head(2)

,ObjectID,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgComp,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,CntryName,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,MatchID,StrucType,StrucDet,IN_SingleLine,USER_id,USER_redcap_event_name,USER_caide_sex,USER_caide_age,USER_caide_education,USER_caide_hypertension,USER_caide_cholesterol,USER_caide_height,USER_caide_height_meters,USER_caide_weight,USER_caide_weight_kg,USER_caide_bmi_calculation,USER_caide_bmi,USER_caide_physical_activity,USER_caide_score,USER_tics_total,USER_faq_total,USER_address,USER_address_geocode,USER_initial_inquiry_v2,USER_waitlist_referral_source,USER_id_MHB,geometry,index_right,GISJOIN,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Leng,Shape_Area
0,1,M,100.0,A,"5241 S Teresa Pt, Homosassa, Florida, 34446","5241 S Teresa Pt, Homosassa, FL, 34446, USA",5241 S Teresa Pt,PointAddress,NaN,NaN,"5241 S Teresa Pt, Homosassa, Florida, 34446",NaN,NaN,20.0,NaN,5241,NaN,NaN,NaN,NaN,S,NaN,Teresa,Pt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5241 S Teresa Pt,NaN,NaN,NaN,NaN,Homosassa,NaN,Citrus County,Florida,FL,NaN,NaN,34446.0,2435.0,USA,United States,ENG,0,-82.527652,28.786060,-82.527387,28.785972,-82.528387,-82.526387,28.784972,28.786972,NaN,AQHAfAEAAdABAJUBBAAAAAAMElNPVVRIIFRFUkVTQSBQT0lOVBUENTI0MWYDRU5H,Residential,Single Family,"5241 South Teresa Point Homosassa, FL 34446",16,screening_ii_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5241 South Teresa Point Homosassa, FL 34446","5241 South Teresa Point Homosassa, FL 34446",14,4.0,16_MHB,POINT (-82.52765 28.78606),42364.0,G12001704515023,12,017,451502,3,120174515023,Block Group 3,G5030,S,8511193.0,0.0,+28.7874652,-082.5345932,19000.009842,8.511193e+06
1,2,M,100.0,A,"410 Miller Ln, Rochester, New York, 14617","410 Miller Ln, Rochester, NY, 14617, USA",410 Miller Ln,PointAddress,NaN,NaN,"410 Miller Ln, Rochester, New York, 14617",NaN,NaN,20.0,NaN,410,NaN,NaN,NaN,NaN,NaN,NaN,Miller,Ln,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,410 Miller Ln,NaN,NaN,NaN,NaN,Rochester,NaN,Monroe County,New York,NY,NaN,NaN,14617.0,4311.0,USA,United States,ENG,0,-77.596270,43.205257,-77.596008,43.205264,-77.597008,-77.595008,43.204264,43.206264,NaN,AQEDkAEAVQgCAJUBBAAAAAAMC01JTExFUiBMQU5FFQM0MTBmA0VORw,NaN,NaN,410 Miller Lane Rochester New York 14617,11,screening_ii_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,410 Miller Lane Rochester New York 14617,410 Miller Lane Rochester New York 14617,14,5.0,11_MHB,POINT (-77.59627 43.20526),133603.0,G36005500106023,36,055,010602,3,360550106023,Block Group 3,G5030,S,666912.0,0.0,+43.2008376,-077.5970437,4188.433808,6.669127e+05


In [32]:
#Merge with walkability index
data_final = data_sj.merge(gdf.drop(columns='geometry'), how='left', left_on='GEOID', right_on='GEOID20', indicator='wbi_merge')

#Again, 1 null address not merging, this is expected
display(data_final[data_final['wbi_merge'] != 'both'])
display(data_final.head(2))

,ObjectID,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgComp,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,CntryName,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,MatchID,StrucType,StrucDet,IN_SingleLine,USER_id,USER_redcap_event_name,USER_caide_sex,USER_caide_age,USER_caide_education,USER_caide_hypertension,USER_caide_cholesterol,USER_caide_height,USER_caide_height_meters,USER_caide_weight,USER_caide_weight_kg,USER_caide_bmi_calculation,USER_caide_bmi,USER_caide_physical_activity,USER_caide_score,USER_tics_total,USER_faq_total,USER_address,USER_address_geocode,USER_initial_inquiry_v2,USER_waitlist_referral_source,USER_id_MHB,geometry,index_right,GISJOIN,STATEFP_x,COUNTYFP_x,TRACTCE_x,BLKGRPCE_x,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Leng,Shape_Area_x,GEOID10,GEOID20,STATEFP_y,COUNTYFP_y,TRACTCE_y,BLKGRPCE_y,CSA,CSA_Name,CBSA,CBSA_Name,Ac_Total,Ac_Water,Ac_Land,Ac_Unpr,TotPop,CountHU,HH,Workers,D2B_E8MIXA,D2A_EPHHM,D3B,D4A,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area_y,wbi_merge
16,17,U,0.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,88,screening_ii_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,88_MHB,POINT (0 0),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


,ObjectID,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgComp,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,CntryName,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,MatchID,StrucType,StrucDet,IN_SingleLine,USER_id,USER_redcap_event_name,USER_caide_sex,USER_caide_age,USER_caide_education,USER_caide_hypertension,USER_caide_cholesterol,USER_caide_height,USER_caide_height_meters,USER_caide_weight,USER_caide_weight_kg,USER_caide_bmi_calculation,USER_caide_bmi,USER_caide_physical_activity,USER_caide_score,USER_tics_total,USER_faq_total,USER_address,USER_address_geocode,USER_initial_inquiry_v2,USER_waitlist_referral_source,USER_id_MHB,geometry,index_right,GISJOIN,STATEFP_x,COUNTYFP_x,TRACTCE_x,BLKGRPCE_x,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Leng,Shape_Area_x,GEOID10,GEOID20,STATEFP_y,COUNTYFP_y,TRACTCE_y,BLKGRPCE_y,CSA,CSA_Name,CBSA,CBSA_Name,Ac_Total,Ac_Water,Ac_Land,Ac_Unpr,TotPop,CountHU,HH,Workers,D2B_E8MIXA,D2A_EPHHM,D3B,D4A,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area_y,wbi_merge
0,1,M,100.0,A,"5241 S Teresa Pt, Homosassa, Florida, 34446","5241 S Teresa Pt, Homosassa, FL, 34446, USA",5241 S Teresa Pt,PointAddress,NaN,NaN,"5241 S Teresa Pt, Homosassa, Florida, 34446",NaN,NaN,20.0,NaN,5241,NaN,NaN,NaN,NaN,S,NaN,Teresa,Pt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5241 S Teresa Pt,NaN,NaN,NaN,NaN,Homosassa,NaN,Citrus County,Florida,FL,NaN,NaN,34446.0,2435.0,USA,United States,ENG,0,-82.527652,28.786060,-82.527387,28.785972,-82.528387,-82.526387,28.784972,28.786972,NaN,AQHAfAEAAdABAJUBBAAAAAAMElNPVVRIIFRFUkVTQSBQT0lOVBUENTI0MWYDRU5H,Residential,Single Family,"5241 South Teresa Point Homosassa, FL 34446",16,screening_ii_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5241 South Teresa Point Homosassa, FL 34446","5241 South Teresa Point Homosassa, FL 34446",14,4.0,16_MHB,POINT (-82.52765 28.78606),42364.0,G12001704515023,12,017,451502,3,120174515023,Block Group 3,G5030,S,8511193.0,0.0,+28.7874652,-082.5345932,19000.009842,8.511193e+06,120174515023,120174515023,12,017,451502,3,None,None,26140,"Homosassa Springs, FL",2103.115790,0.0,2103.115790,2103.115790,2687.0,1445.0,1112.0,837.0,0.386358,0.304734,26.892518,-99999.00,5.0,5.0,7.0,1.0,4.333333,19000.009920,8.511193e+06,both
1,2,M,100.0,A,"410 Miller Ln, Rochester, New York, 14617","410 Miller Ln, Rochester, NY, 14617, USA",410 Miller Ln,PointAddress,NaN,NaN,"410 Miller Ln, Rochester, New York, 14617",NaN,NaN,20.0,NaN,410,NaN,NaN,NaN,NaN,NaN,NaN,Miller,Ln,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,410 Miller Ln,NaN,NaN,NaN,NaN,Rochester,NaN,Monroe County,New York,NY,NaN,NaN,14617.0,4311.0,USA,United States,ENG,0,-77.596270,43.205257,-77.596008,43.205264,-77.597008,-77.595008,43.204264,43.206264,NaN,AQEDkAEAVQgCAJUBBAAAAAAMC01JTExFUiBMQU5FFQM0MTBmA0VORw,NaN,NaN,410 Miller Lane Rochester New York 14617,11,screening_ii_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,410 Miller Lane Rochester New York 14617,410 Miller Lane Rochester New York 14617,14,5.0,11_MHB,POINT (-77.59627 43.20526),133603.0,G36005500106023,36,055,010602,3,360550106023,Block Group 3,G5030,S,666912.0,0.0,+43.2008376,-077.5970437,4188.433808,6.669127e+05,360550106023,360550106023,36,055,010602,3,464,"Rochester-Batavia-Seneca Falls, NY",40380,"Rochester, NY",164.793955,0.0,164.793955,164.793955,775.0,500.0,476.0,419.0,0.678828,0.819559,68.639411,811.38,19.0,15.0,11.0,14.0,14.000000,4188.433784,6.669127e+05,both


In [36]:
data_final.filter(regex='MHB|GEOID|Nat').head(2)

,USER_id_MHB,GEOID,GEOID10,GEOID20,NatWalkInd
0,16_MHB,120174515023,120174515023,120174515023,4.333333
1,11_MHB,360550106023,360550106023,360550106023,14.000000


In [37]:
data_final.to_csv('MHB_Walkability_Analytic.csv', index=False)